In [1]:
import pandas as pd
import joblib
import os

MODELS_DIR = os.path.join(os.getcwd(), "../backend/models")

In [2]:
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)

In [3]:
df = pd.read_csv("../cases/case_2/dataset2.zip")
df.head()

,Product ID,Product Title,Merchant ID,Cluster ID,Cluster Label,Category ID,Category Label
0,1,apple iphone 8 plus 64gb silver,1,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
1,2,apple iphone 8 plus 64 gb spacegrau,2,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
2,3,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...,3,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
3,4,apple iphone 8 plus 64gb space grey,4,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
4,5,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,5,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones


In [4]:
df[" Category Label"].value_counts()

Fridge Freezers     5501
Mobile Phones       4081
Washing Machines    4044
CPUs                3862
Fridges             3584
TVs                 3564
Dishwashers         3424
Digital Cameras     2697
Microwaves          2342
Freezers            2212
Name:  Category Label, dtype: int64

In [5]:
df[" Cluster Label"].value_counts()

Canon IXUS 185              27
Samsung UE49NU7100          24
Canon PowerShot SX730 HS    24
Samsung UE65NU7100          23
Canon IXUS 285 HS           23
                            ..
Beko WDX8543130W             1
LG F1496TDA                  1
Logik L612WM16               1
Bosch WAN28280GB             1
Siemens WM14T470GB           1
Name:  Cluster Label, Length: 12849, dtype: int64

In [6]:
for idx, row in df[[" Category ID", " Category Label"]].drop_duplicates().iterrows():
    print(f'({row[" Category ID"]},"{row[" Category Label"]}"),', sep=",")

(2612,"Mobile Phones"),
(2614,"TVs"),
(2615,"CPUs"),
(2617,"Digital Cameras"),
(2618,"Microwaves"),
(2619,"Dishwashers"),
(2620,"Washing Machines"),
(2621,"Freezers"),
(2622,"Fridge Freezers"),
(2623,"Fridges"),


# Model on first word

In [7]:
df["first_word"] = df[" Cluster Label"].str.split().str[0]
df["first_word"].nunique()

284

In [8]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

X = df["first_word"]
y = df[" Category ID"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

pipleline = Pipeline(
    [
        (
            "encoder",
            OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=int(1e9)),
        ),
        ("model", RandomForestClassifier()),
    ]
)

/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
pipleline.fit(pd.DataFrame(X_train), y_train)

pipleline.score(pd.DataFrame(X_test), y_test)

0.5459436500070791

In [10]:
print(classification_report(y_test, pipleline.predict(pd.DataFrame(X_test))))

              precision    recall  f1-score   support

        2612       1.00      0.74      0.85       818
        2614       0.53      0.88      0.66       723
        2615       1.00      1.00      1.00       771
        2617       0.83      0.80      0.82       542
        2618       0.60      0.32      0.41       469
        2619       0.26      0.55      0.36       662
        2620       0.39      0.50      0.44       796
        2621       0.43      0.11      0.17       439
        2622       0.37      0.38      0.37      1115
        2623       0.44      0.04      0.07       728

    accuracy                           0.55      7063
   macro avg       0.59      0.53      0.52      7063
weighted avg       0.58      0.55      0.53      7063



In [11]:
joblib.dump(pipleline, os.path.join(MODELS_DIR, "base.joblib"))

['/home/samoed/Desktop/itmo-ai-meta/ml-service/experiments/../backend/models/base.joblib']

# TF-iDF + Logreg on cluster label

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [13]:
X = df[" Cluster Label"]
y = df[" Category ID"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [14]:
pipe = Pipeline(
    [
        ("tfidf", TfidfVectorizer()),
        ("clf", LogisticRegression(random_state=42)),
    ]
)


pipe.fit(X_train, y_train)

/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LogisticRegression(random_state=42))])

In [15]:
preditions = pipe.predict(X_test)

print(classification_report(y_test, preditions))

              precision    recall  f1-score   support

        2612       1.00      0.97      0.99       818
        2614       0.93      0.99      0.96       723
        2615       1.00      1.00      1.00       771
        2617       1.00      0.99      1.00       542
        2618       0.89      0.89      0.89       469
        2619       0.80      0.80      0.80       662
        2620       0.93      0.96      0.95       796
        2621       0.91      0.78      0.84       439
        2622       0.82      0.87      0.84      1115
        2623       0.88      0.82      0.85       728

    accuracy                           0.91      7063
   macro avg       0.91      0.91      0.91      7063
weighted avg       0.91      0.91      0.91      7063



In [16]:
joblib.dump(pipleline, os.path.join(MODELS_DIR, "logreg_tfidf.joblib"))

['/home/samoed/Desktop/itmo-ai-meta/ml-service/experiments/../backend/models/logreg_tfidf.joblib']

# Catboost on ids

In [17]:
from catboost import CatBoostClassifier

In [18]:
X = pd.DataFrame(df[" Cluster Label"])
y = df[" Category ID"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

cat = CatBoostClassifier(
    iterations=10,
    learning_rate=0.1,
    random_seed=42,
    logging_level="Silent",
    task_type="GPU",
    text_features=[" Cluster Label"],
    # cat_features=[" Merchant ID", " Cluster ID"],
)

cat.fit(X_train, y_train)

/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [19]:
pred = cat.predict(X_test)

print(classification_report(y_test, pred))

              precision    recall  f1-score   support

        2612       1.00      0.97      0.98       818
        2614       0.90      1.00      0.95       723
        2615       1.00      1.00      1.00       771
        2617       1.00      0.99      0.99       542
        2618       0.84      0.71      0.77       469
        2619       0.67      0.51      0.58       662
        2620       0.94      0.94      0.94       796
        2621       0.87      0.54      0.66       439
        2622       0.67      0.76      0.71      1115
        2623       0.58      0.75      0.66       728

    accuracy                           0.83      7063
   macro avg       0.85      0.82      0.82      7063
weighted avg       0.84      0.83      0.83      7063



In [20]:
cat.save_model(os.path.join(MODELS_DIR, "catboost.model"))